# **Welcome to Before You Train- Data preprocessing**
###**By [Dina Bavli](https://www.linkedin.com/in/dina-bavli-502430158/)**

In this session we will learn ways for getting to know your data and how to handle missing data, outliers and floats transformation.

####**Data is based on the IEEE-CIS [Fraud](https://www.kaggle.com/c/ieee-fraud-detection/overview) Detection Kaggle competition dataset.**



## **You shouldn't run all of this notebook at once!**
After your first `fillna` you won't have any more missing data to fill.

You can always save what ever version you like and start over for another version using this syntax:


```
#Connect to drive
from google.colab import drive 
drive.mount('/content/gdrive')

#Saving to drive
df_trans.to_csv('gdrive/My Drive/Colab Notebooks/fillna_dist_trans_outclip.csv',index=False,compression='gzip')


#Reading from drive
df_trans=pd.read_csv('gdrive/My Drive/Colab Notebooks/fillna_dist_trans_outclip.csv',compression='gzip')


```
**Enjoy :0)** 

![alt text](https://media.makeameme.org/created/hard-to-handle-5a99c3.jpg)





#First Thing First

##Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from numpy import mean
from numpy import std
import math
import scipy
from scipy.stats import stats
from numpy.random import choice
import random
from natsort import humansorted
from scipy.stats import kurtosistest
from scipy.stats import kurtosis



##Loading The Data

In [ ]:
# Script used to download data from Google Drive
!wget https://gist.githubusercontent.com/guysmoilov/ff68ef3416f99bd74a3c431b4f4c739a/raw/4587c1306e00a576037379ee410d6c35f3daada6/gdrive_download.sh

# Download from https://drive.google.com/uc?export=download&id=1vFNmOtuP0kMbYOlOITkgdfwK1YwLlYv1
!bash -c "source gdrive_download.sh && gdrive_download 1vFNmOtuP0kMbYOlOITkgdfwK1YwLlYv1 ./fraud_data_base.csv.gz"

# Unzip it
!gunzip ./fraud_data_base.csv.gz

--2020-02-24 17:51:13--  https://gist.githubusercontent.com/guysmoilov/ff68ef3416f99bd74a3c431b4f4c739a/raw/4587c1306e00a576037379ee410d6c35f3daada6/gdrive_download.sh
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 434 [text/plain]
Saving to: ‘gdrive_download.sh.2’

gdrive_download.sh. 100%[===================>]     434  --.-KB/s    in 0s      

2020-02-24 17:51:13 (133 MB/s) - ‘gdrive_download.sh.2’ saved [434/434]

--2020-02-24 17:51:14--  https://drive.google.com/uc?export=download&confirm=yJCp&id=1vFNmOtuP0kMbYOlOITkgdfwK1YwLlYv1
Resolving drive.google.com (drive.google.com)... 74.125.31.139, 74.125.31.101, 74.125.31.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.139|:443... connected.
HTTP request sent, awaiting response... 302 Mov

##Seting Up

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('./fraud_data_base.csv')
df

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,...,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
0,0,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,NaN,NaN,NaN,NaN,NaN,1,12
1,0,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,google,com,NaN,NaN,NaN,1,12
2,0,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,microsoft,com,NaN,NaN,NaN,1,12
3,0,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,yahoo,com,NaN,NaN,NaN,1,12
4,0,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android,Samsung,32.0,2220x1080,match_status:2,T,F,T,T,mobile,5,0,2,google,com,NaN,NaN,Samsung,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,56.0,56.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,NaN,NaN,NaN,NaN,NaN,1,6
590536,0,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590537,0,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590538,0,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,3.0,NaN,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,1.0,5.0,1.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,22.0,22.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,aol,com,NaN,NaN,NaN,1,6


In [ ]:
df.columns

Index(['isFraud', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3',
       'card4', 'card5', 'card6', 'addr1',
       ...
       '_Weekdays', '_Hours', '_Days', 'P_emaildomain_bin',
       'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix',
       'device_name', 'had_id', '_Month'],
      dtype='object', length=439)

## Set Columns Type

In [ ]:
cat_cols=['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
       'id_12', 'id_13', 'id_15', 'id_16', 'id_17', 'id_19', 'id_20', 'id_28',
       'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       '_Weekdays', '_Hours', '_Days', 'P_emaildomain_bin',
       'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix',
       'device_name', 'had_id', '_Month']

In [ ]:
for i in range(len(cat_cols)):
  col = cat_cols[i]
  df[col] = df[col].astype(str) 

In [ ]:
object_cols = df.select_dtypes(include = ['object']).columns
len(object_cols)

48

In [ ]:
numeric_cols = df.select_dtypes(exclude = ['object'])

In [ ]:
int_cols = set(df.select_dtypes(include = ['integer']).columns)
int_cols

{'isFraud'}

In [ ]:
int_cols.remove('isFraud')

In [ ]:
df_integers= df[list(int_cols)]

In [ ]:
float_cols = df.select_dtypes(include=['floating']).columns
len(float_cols)

390

In [ ]:
float_to_int_cols = set()
for col in float_cols:
  col_df = df[col].dropna()
  col_should_be_int = col_df.map(float.is_integer).all() 
  if col_should_be_int:
    float_to_int_cols.add(col)

In [ ]:
len(float_to_int_cols)

310

In [ ]:
 int_cols = int_cols.union(float_to_int_cols)
 len(int_cols)

310

# Missing Data

## Knowing it's been missing before handling

In [ ]:
col_name = df.columns #Columns Names
df2 = df.isnull() #New datafram- is null? true or false?

# New names for the new columns:
new_col = [col.replace(col, col+str(' isNAN')) for col in col_name]

df2.columns = new_col #Naming the new columns

In [ ]:
drop_isNAN = df2.join(df, how='outer') #Joining the columns

In [ ]:
drop_isNAN.columns

Index(['C1 isNAN', 'C10 isNAN', 'C11 isNAN', 'C12 isNAN', 'C13 isNAN',
       'C14 isNAN', 'C2 isNAN', 'C3 isNAN', 'C4 isNAN', 'C5 isNAN',
       ...
       'id_19', 'id_20', 'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37',
       'id_38', 'isFraud'],
      dtype='object', length=720)

In [ ]:
drop_isNAN = drop_isNAN.reindex(columns=humansorted(drop_isNAN.columns))# Reindexing the columns order

In [ ]:
drop_isNAN.columns

Index(['addr1', 'addr1 isNAN', 'addr2', 'addr2 isNAN', 'card1', 'card1 isNAN',
       'card2', 'card2 isNAN', 'card3', 'card3 isNAN',
       ...
       'V317', 'V317 isNAN', 'V318', 'V318 isNAN', 'V319', 'V319 isNAN',
       'V320', 'V320 isNAN', 'V321', 'V321 isNAN'],
      dtype='object', length=720)

##Dropping Missing Data



```
#Drop the rows where at least one element is missing.
df.dropna() 
#Drop the columns where at least one element is missing.
df.dropna(axis='columns') 
#Drop the rows where all elements are missing.
df.dropna(how='all')
#Keep only the rows with at least 2 non-NA values.
df.dropna(thresh=2)
#Define in which columns to look for missing values.
df.dropna(subset=['name', 'born'])
#Keep the DataFrame with valid entries in the same variable.
df.dropna(inplace=True)
```



In [ ]:
missing_rat = [col for col in df.columns if df[col].isnull().sum() /
               df.shape[0] > 0.8]

In [ ]:
cols_to_drop = list(missing_rat)
null_drop = df[df.columns.difference(cols_to_drop)]

## Fill with 'NaN' and '0'

In [ ]:
train2 = df.copy()
for col in object_cols:
    train2[col] = train2[col].fillna('NaN')

NameError: ignored

In [ ]:
for col in numeric_cols:
    train2[col] = train2[col].fillna(0)

##Forward and Back -Fill




```
# forward-fill
df.fillna(method = 'ffill')
# back-fill
df.fillna(method = 'bfill')
```


##Fill Nulls with Mode and Mean

In [ ]:
for col in object_cols:
  df[col].fillna(df[col].mode()[0], inplace = True)

In [ ]:
for col in int_cols:
  value_to_fill  =round(df[col].mean())
  #print(col,value_to_fill)
  df[col].fillna(value_to_fill, inplace = True)

In [ ]:
for col in float_cols:
  value_to_fill = df[col].mean()
  #print(col,value_to_fill)
  df[col].fillna(value_to_fill,inplace = True)

##Fill Nulls by Distribution


In [ ]:
for col in int_cols:
  values=df_integers[col].unique().tolist()
  values__probs = df_integers.groupby(col).size().div(len(df_integers))
  randomnumber = choice(values, p=values__probs)
  df[col] = df[col].fillna(randomnumber)

In [ ]:
for col in float_cols:
    vals=pd.Series(df[col][df[col].notnull()])
    density = scipy.stats.gaussian_kde(vals)
    df_nan=df.loc[(df[col].isnull())]
    indexes=set(df_nan.index)
    sample=density.resample(len(indexes)).T[:,0]
    values_list=random.sample(set(pd.Series(sample)),k=len(indexes))
    values_to_fill={}
    n=0;
    for i in indexes:
        values_to_fill[i]=values_list[n]
        n+=1

    df[col] = df[col].fillna(value=values_to_fill)

##Handle Nulls with Interpolate

**Syntax**: DataFrame.interpolate(method=’linear’, axis=0, limit=None, inplace=False, 
limit_direction=’forward’, limit_area=None, downcast=None, **kwargs)

**Parameters** :
method : {‘linear’, ‘time’, ‘index’, ‘values’, ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘barycentric’, ‘krogh’, ‘polynomial’, ‘spline’, ‘piecewise_polynomial’, ‘from_derivatives’, ‘pchip’, ‘akima’}

**axis** : 0 fill column-by-column and 1 fill row-by-row.

**limit** : Maximum number of consecutive NaNs to fill. Must be greater than 0.

**limit_direction** : {‘forward’, ‘backward’, ‘both’}, default ‘forward’

**limit_area** : None (default) no fill restriction. inside Only fill NaNs surrounded by valid values (interpolate). outside Only fill NaNs outside valid values (extrapolate). If limit is specified, consecutive NaNs will be filled in this direction.
inplace : Update the NDFrame in place if possible.

**downcast** : Downcast dtypes if possible.
kwargs : keyword arguments to pass on to the interpolating function.

**Returns** : Series or DataFrame of same shape interpolated at the NaNs

In [ ]:
df.interpolate(method='linear', limit_direction='forward', axis=0)

# Transformation for Floats

In [ ]:
df_numeric=df._get_numeric_data()
df_describe=df_numeric.describe()
df_describe = df_describe.T

In [ ]:
dtype_df=df_numeric.dtypes
df_describe['dtypes']=dtype_df
Cardinality=df_numeric.apply(pd.Series.nunique)
df_describe['Cardinality']=Cardinality
df_skew=df_numeric.skew(axis = 0, skipna = True)
df_describe['Skewtest']=df_skew

In [ ]:
df_describe.round(2)

,count,mean,std,min,25%,50%,75%,max,dtypes,Cardinality,Skewtest
isFraud,590540.0,0.03,0.18,0.00,0.00,0.00,0.00,1.00,int64,2,5.06
TransactionDT,590540.0,7372311.31,4617223.65,86400.00,3027057.75,7306527.50,11246620.00,15811131.00,int64,573349,0.13
TransactionAmt,590540.0,135.03,239.16,0.25,43.32,68.77,125.00,31937.39,float64,20902,14.37
card1,590540.0,9898.73,4901.17,1000.00,6019.00,9678.00,14184.00,18396.00,int64,13553,-0.04
card2,590540.0,362.56,156.60,100.00,215.00,361.00,512.00,600.00,float64,501,-0.20
...,...,...,...,...,...,...,...,...,...,...,...
id_11,590540.0,99.75,0.55,90.00,99.75,99.75,99.75,100.00,float64,366,-9.88
id_13,590540.0,48.05,5.47,10.00,48.05,48.05,48.05,64.00,float64,55,-2.70
id_17,590540.0,189.45,14.76,100.00,189.45,189.45,189.45,229.00,float64,105,0.28
id_19,590540.0,353.13,68.53,100.00,353.13,353.13,353.13,671.00,float64,523,0.41


In [ ]:
# return all columns with cardinality more than threshold and pass the skew test
def get_cols_for_skew_test(df,threshold):
  cols_for_skew_test = df_describe[df_describe['Cardinality']>threshold].index
  print('There are ' + str(len(cols_for_skew_test)) + 
        ' columns with cardinality higher than '+ str(threshold))
  
  is_skew_cols=set()
  for col in cols_for_skew_test:
    skewness, pvalue = stats.skewtest(df_numeric[col], 
                                      axis = 0, nan_policy = 'omit')
    if pvalue<0.01:
      is_skew_cols.add(col)
  
  print('Out of them, there are ' + str(len(is_skew_cols)) +
        ' passed the skew test')  
  print(list(is_skew_cols))
  return set(is_skew_cols)

In [ ]:
skew_cols = get_cols_for_skew_test(df_describe,10)

In [ ]:
pos_skew_cols = skew_cols.intersection(set(df_describe
                                           [df_describe['Skewtest']>0].index))
print(len(pos_skew_cols))
print(sorted(pos_skew_cols))

In [ ]:
#This loop is taking time... please be patient :)
for col in pos_skew_cols:
  #print(col)
  min_val = df_describe['min'][col]
  #print('min: ' + str(min_val))
  df[col]=df[col].apply(lambda x: np.log(x+abs(min_val)+0.001))

In [ ]:
neg_skew_cols = skew_cols.intersection(set(df_describe
                                           [df_describe['Skewtest']<0].index))
print(len(neg_skew_cols))
print(sorted(neg_skew_cols))

In [ ]:
df[list(neg_skew_cols)]=df[list(neg_skew_cols)].apply(lambda x: x*x)

#Outliers

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    
    if iqr>0:
      kurt_value=kurtosis(df_in[col_name])
      if(abs(kurt_value)<4):
        #print(col)
        statistic,p_value = kurtosistest(df_in[col_name])
        df_out = df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
        print(col_name,"cardinality: "+str(df_in[col_name].nunique()),"iqr:" +str(iqr),"kurtosis value "+str(kurt_value),"#outliers:"+str(len(set(df_out.index))),"(fence_low,fence_high): (" +str(fence_low)+","+str(fence_high)+")")
        #print("(statistic,p_value) :", "("+str(statistic)+','+str(p_value)+")")
        return set(df_out.index)
    return set()

In [ ]:
outliers=set()
for col in df_numeric.columns:
  outliers_indexes=remove_outlier(df_numeric,col)
  outliers=outliers.union(outliers_indexes)
print('There are total ' +str(len(outliers))+' outliers')

D1 cardinality: 641 iqr:489.0 kurtosis value 2.209491302472257 #outliers:0 (fence_low,fence_high): (-733.5,1222.5)
D2 cardinality: 641 iqr:479.0 kurtosis value 2.3890509945768805 #outliers:0 (fence_low,fence_high): (-715.5,1200.5)
D4 cardinality: 808 iqr:516.0 kurtosis value 1.1576985459069524 #outliers:0 (fence_low,fence_high): (-774.0,1290.0)
D10 cardinality: 818 iqr:522.0 kurtosis value 1.1534771814585731 #outliers:0 (fence_low,fence_high): (-783.0,1305.0)
D11 cardinality: 676 iqr:469.0 kurtosis value 2.201553466575306 #outliers:0 (fence_low,fence_high): (-703.5,1172.5)
D15 cardinality: 859 iqr:553.0 kurtosis value -0.08944451812093446 #outliers:0 (fence_low,fence_high): (-829.5,1382.5)
V10 cardinality: 5 iqr:1.0 kurtosis value 0.8021323098948949 #outliers:187 (fence_low,fence_high): (-1.5,2.5)
V11 cardinality: 6 iqr:1.0 kurtosis value 3.057634958791426 #outliers:800 (fence_low,fence_high): (-1.5,2.5)
V12 cardinality: 4 iqr:1.0 kurtosis value -1.4439170579529321 #outliers:120 (fence

In [ ]:
#Remove outliers
df_out= train.drop(train.index[list(outliers)])

##Some Thing Extra

In [ ]:
data_isFraud_1=df[df['isFraud']==1]
data_isFraud_0=df[df['isFraud']==0]

print('data_isFraud_1 shape: ',data_isFraud_1.shape)
print('data_isFraud_0 shape: ',data_isFraud_0.shape)


data_isFraud_1 shape:  (20663, 439)
data_isFraud_0 shape:  (569877, 439)
